<a href="https://colab.research.google.com/github/udlbook/udlbook/blob/main/Notebooks/Chap12/12_1_Self_Attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Notebook 12.1: Self Attention**

This notebook builds a self-attention mechanism from scratch, as discussed in section 12.2 of the book.

Work through the cells below, running each cell in turn. In various places you will see the words "TO DO". Follow the instructions at these places and make predictions about what is going to happen or write code to complete the functions.

Contact me at udlbookmail@gmail.com if you find any mistakes or have any suggestions.



In [21]:
import numpy as np
import matplotlib.pyplot as plt

The self-attention mechanism maps $N$ inputs $\mathbf{x}_{n}\in\mathbb{R}^{D}$ and returns $N$ outputs $\mathbf{x}'_{n}\in \mathbb{R}^{D}$.  



In [22]:
# Set seed so we get the same random numbers
np.random.seed(3)
# Number of inputs
N = 3
# Number of dimensions of each input
D = 4
# Create an empty list
all_x = []
# Create elements x_n and append to list
for n in range(N):
  all_x.append(np.random.normal(size=(D,1)))
# Print out the list
print(all_x)

[array([[ 1.78862847],
       [ 0.43650985],
       [ 0.09649747],
       [-1.8634927 ]]), array([[-0.2773882 ],
       [-0.35475898],
       [-0.08274148],
       [-0.62700068]]), array([[-0.04381817],
       [-0.47721803],
       [-1.31386475],
       [ 0.88462238]])]


We'll also need the weights and biases for the keys, queries, and values (equations 12.2 and 12.4)

In [23]:
# Set seed so we get the same random numbers
np.random.seed(0)

# Choose random values for the parameters
omega_q = np.random.normal(size=(D,D))
omega_k = np.random.normal(size=(D,D))
omega_v = np.random.normal(size=(D,D))
beta_q = np.random.normal(size=(D,1))
beta_k = np.random.normal(size=(D,1))
beta_v = np.random.normal(size=(D,1))

Now let's compute the queries, keys, and values for each input

In [24]:
# Make three lists to store queries, keys, and values
all_queries = []
all_keys = []
all_values = []

# For every input
for x in all_x:
  # TODO -- compute the keys, queries and values.

  # Replace these three lines

  query = np.dot(omega_q, x) + beta_q
  key = np.dot(omega_k, x) + beta_k
  value = np.dot(omega_v, x) + beta_v

  all_queries.append(query)
  all_keys.append(key)
  all_values.append(value)

We'll need a softmax function (equation 12.5) -- here, it will take a list of arbitrary numbers and return a list where the elements are non-negative and sum to one


In [25]:
def softmax(items_in):
  # Convert the list of arrays to a single NumPy array
  items_in = np.array(items_in)

  # TODO Compute the elements of items_out
  # Replace this line
  items_out = items_in.copy()
  # COMPUTE SOFTMAX
  items_out = np.exp(items_out, out=items_out) #Now it shoudl work because items_out is a numpy array

  items_out /= np.sum(items_out)

  return items_out

Now compute the self attention values:

In [26]:
# Assuming all_queries, all_keys, all_values, N, D are defined

# Create empty list for output
all_x_prime = []

# For each output
for n in range(N):
    # Create list for dot products of query N with all keys
    all_km_qn = []
    # Compute the dot products
    for key in all_keys:
        # Compute the appropriate dot product using the current query
        dot_product = np.dot(all_queries[n].T, key)
        # Store dot product
        all_km_qn.append(dot_product)

    # Compute dot product
    attention = softmax(all_km_qn)

    # Reshape attention to a 1D array for element-wise multiplication
    attention = attention.reshape(-1)

    # Print result (should be positive sum to one)
    print("Attentions for output ", n)
    print(attention)

    # Compute a weighted sum of all of the values according to the attention
    x_prime = np.zeros((D, 1))
    for i in range(len(all_values)):
        x_prime += attention[i] * all_values[i]

    all_x_prime.append(x_prime)

# Print out true values to check you have it correct
print("x_prime_0_calculated:", all_x_prime[0].transpose())
print("x_prime_0_true: [[ 0.94744244 -0.24348429 -0.91310441 -0.44522983]]")
print("x_prime_1_calculated:", all_x_prime[1].transpose())
print("x_prime_1_true: [[ 1.64201168 -0.08470004  4.02764044  2.18690791]]")
print("x_prime_2_calculated:", all_x_prime[2].transpose())
print("x_prime_2_true: [[ 1.61949281 -0.06641533  3.96863308  2.15858316]]")

Attentions for output  0
[1.24326146e-13 9.98281489e-01 1.71851130e-03]
Attentions for output  1
[2.79525306e-12 5.85506360e-03 9.94144936e-01]
Attentions for output  2
[0.00505708 0.00654776 0.98839516]
x_prime_0_calculated: [[ 0.94744244 -0.24348429 -0.91310441 -0.44522983]]
x_prime_0_true: [[ 0.94744244 -0.24348429 -0.91310441 -0.44522983]]
x_prime_1_calculated: [[ 1.64201168 -0.08470004  4.02764044  2.18690791]]
x_prime_1_true: [[ 1.64201168 -0.08470004  4.02764044  2.18690791]]
x_prime_2_calculated: [[ 1.61949281 -0.06641533  3.96863308  2.15858316]]
x_prime_2_true: [[ 1.61949281 -0.06641533  3.96863308  2.15858316]]


Now let's compute the same thing, but using matrix calculations.  We'll store the $N$ inputs $\mathbf{x}_{n}\in\mathbb{R}^{D}$ in the columns of a $D\times N$ matrix, using equations 12.6 and 12.7/8.

Note:  The book uses column vectors (for compatibility with the rest of the text), but in the wider literature it is more normal to store the inputs in the rows of a matrix;  in this case, the computation is the same, but all the matrices are transposed and the operations proceed in the reverse order.

In [27]:
# Define softmax operation that works independently on each column
def softmax_cols(data_in):
  # Exponentiate all of the values
  exp_values = np.exp(data_in) ;
  # Sum over columns
  denom = np.sum(exp_values, axis = 0);
  # Replicate denominator to N rows
  denom = np.matmul(np.ones((data_in.shape[0],1)), denom[np.newaxis,:])
  # Compute softmax
  softmax = exp_values / denom
  # return the answer
  return softmax

In [28]:
def self_attention(X, omega_v, omega_q, omega_k, beta_v, beta_q, beta_k):
    """
    Computes self-attention in matrix form.

    Args:
        X: Input matrix of shape (D, N) where D is the feature dimension and N is the number of inputs.
        omega_v, omega_q, omega_k: Weight matrices for values, queries, and keys, respectively.
        beta_v, beta_q, beta_k: Bias vectors for values, queries, and keys, respectively.

    Returns:
        X_prime: Output matrix of shape (D, N) after applying self-attention.
    """

    # 1. Compute queries, keys, and values
    Queries = omega_q @ X + beta_q  # (D, N)
    Keys = omega_k @ X + beta_k      # (D, N)
    Values = omega_v @ X + beta_v    # (D, N)

    # 2. Compute dot products (unnormalized attention weights)
    # Note: Transposing Keys to (N, D) for matrix multiplication
    # Resulting shape: (N, N) - attention weights for each input with all other inputs
    Unnormalized_attention_weights = Queries.T @ Keys

    # 3. Apply softmax to calculate attentions (row-wise)
    # Dividing each row by its sum to normalize
    Attentions = np.apply_along_axis(softmax, 1, Unnormalized_attention_weights)

    # 4. Weight values by attentions to get the final output
    # Resulting shape: (D, N)
    X_prime = Values @ Attentions.T

    return X_prime

In [29]:
# Copy data into matrix
X = np.zeros((D, N))
X[:,0] = np.squeeze(all_x[0])
X[:,1] = np.squeeze(all_x[1])
X[:,2] = np.squeeze(all_x[2])

# Run the self attention mechanism
X_prime = self_attention(X,omega_v, omega_q, omega_k, beta_v, beta_q, beta_k)

# Print out the results
print(X_prime)

[[ 0.94744244  1.64201168  1.61949281]
 [-0.24348429 -0.08470004 -0.06641533]
 [-0.91310441  4.02764044  3.96863308]
 [-0.44522983  2.18690791  2.15858316]]


If you did this correctly, the values should be the same as above.

TODO:  

Print out the attention matrix
You will see that the values are quite extreme (one is very close to one and the others are very close to zero.  Now we'll fix this problem by using scaled dot-product attention.

In [30]:
def scaled_dot_product_self_attention(X, omega_v, omega_q, omega_k, beta_v, beta_q, beta_k):
    """
    Computes scaled dot-product self-attention.

    Args:
        X: Input matrix of shape (D, N) where D is the feature dimension and N is the number of inputs.
        omega_v, omega_q, omega_k: Weight matrices for values, queries, and keys, respectively.
        beta_v, beta_q, beta_k: Bias vectors for values, queries, and keys, respectively.

    Returns:
        X_prime: Output matrix of shape (D, N) after applying scaled dot-product self-attention.
    """

    # 1. Compute queries, keys, and values
    Queries = omega_q @ X + beta_q  # (D, N)
    Keys = omega_k @ X + beta_k      # (D, N)
    Values = omega_v @ X + beta_v    # (D, N)

    # 2. Compute dot products (unnormalized attention weights)
    Unnormalized_attention_weights = Queries.T @ Keys  # (N, N)

    # 3. Scale the dot products by the square root of the key dimension (equation 12.9)
    d_k = Keys.shape[0]  # Dimension of the keys
    Scaled_attention_weights = Unnormalized_attention_weights / np.sqrt(d_k)

    # 4. Apply softmax to calculate attentions (row-wise)
    Attentions = np.apply_along_axis(softmax, 1, Scaled_attention_weights)

    # 5. Weight values by attentions to get the final output
    X_prime = Values @ Attentions.T  # (D, N)

    return X_prime

In [31]:
# Run the self attention mechanism
X_prime = scaled_dot_product_self_attention(X,omega_v, omega_q, omega_k, beta_v, beta_q, beta_k)

# Print out the results
print(X_prime)

[[ 0.97411966  1.59622051  1.32638014]
 [-0.23738409 -0.09516106  0.13062402]
 [-0.72333202  3.70194096  3.02371664]
 [-0.34413007  2.01339538  1.6902419 ]]
